# Lab 4.1.1: Vision-Language Models - SOLUTIONS

This notebook contains complete solutions for all exercises and challenges from Lab 01.

---

In [ ]:
# Setup (same as main notebook)
import gc
import time
import requests
from io import BytesIO
from typing import Optional, Union, List, Dict

import torch
from PIL import Image
import matplotlib.pyplot as plt

from transformers import AutoProcessor, LlavaForConditionalGeneration

# Load model
model_name = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_name)
model = LlavaForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)

def load_image(source: Union[str, str]) -> Image.Image:
    source_str = str(source)
    if source_str.startswith(("http://", "https://")):
        response = requests.get(source_str, timeout=10)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(source_str)
    return image.convert("RGB")

def analyze_image(image: Image.Image, question: str, max_new_tokens: int = 256) -> str:
    prompt = f"USER: <image>\n{question}\nASSISTANT:"
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    
    response = processor.decode(output_ids[0], skip_special_tokens=True)
    if "ASSISTANT:" in response:
        response = response.split("ASSISTANT:")[-1].strip()
    return response

print("✅ Setup complete!")

---

## Challenge Solution: Image Comparison Assistant

Build an assistant that compares two images.

In [ ]:
def compare_images(
    image1: Image.Image,
    image2: Image.Image,
    purpose: str = "general"
) -> Dict[str, str]:
    """
    Compare two images and return analysis.
    
    Args:
        image1: First image
        image2: Second image
        purpose: What the images are being compared for
        
    Returns:
        Dictionary with similarities, differences, and recommendation
    """
    # Analyze each image individually first
    desc1 = analyze_image(image1, "Describe this image in detail in 2-3 sentences.")
    desc2 = analyze_image(image2, "Describe this image in detail in 2-3 sentences.")
    
    # Create a combined analysis prompt
    # Note: LLaVA can only see one image at a time, so we use descriptions
    comparison_prompt = f"""I have two images:

Image 1: {desc1}

Image 2: {desc2}

Compare these two images. What are the similarities and differences?"""
    
    # Get similarities
    similarities = analyze_image(
        image1,  # Use first image as visual context
        f"Based on this image and the description '{desc2}' of another image, what might be similar between them?",
        max_new_tokens=150
    )
    
    # Get differences
    differences = analyze_image(
        image1,
        f"Based on this image and the description '{desc2}' of another image, what are the key differences?",
        max_new_tokens=150
    )
    
    # Get recommendation based on purpose
    recommendation_prompt = f"""Comparing two images:
Image 1: {desc1}
Image 2: {desc2}

For the purpose of '{purpose}', which image would be better and why? Give a brief recommendation."""
    
    recommendation = analyze_image(
        image1,
        recommendation_prompt,
        max_new_tokens=150
    )
    
    return {
        "image1_description": desc1,
        "image2_description": desc2,
        "similarities": similarities,
        "differences": differences,
        "recommendation": recommendation,
    }

In [ ]:
# Test the comparison
img1_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg"
img2_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/YellowLabradorLooking_new.jpg/1200px-YellowLabradorLooking_new.jpg"

img1 = load_image(img1_url)
img2 = load_image(img2_url)

# Display images
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(img1)
axes[0].set_title("Image 1")
axes[0].axis('off')
axes[1].imshow(img2)
axes[1].set_title("Image 2")
axes[1].axis('off')
plt.tight_layout()
plt.show()

# Compare
print("\n🔍 Comparing images...")
result = compare_images(img1, img2, purpose="choosing a family pet")

print("\n📊 Comparison Results:")
print("=" * 60)
for key, value in result.items():
    print(f"\n{key.upper().replace('_', ' ')}:")
    print(f"  {value}")

---

## Alternative Approach: Side-by-Side Grid

If you want to show both images to the VLM at once, you can create a combined image grid.

In [ ]:
def create_comparison_image(
    image1: Image.Image,
    image2: Image.Image,
    labels: tuple = ("Image 1", "Image 2"),
    padding: int = 20,
) -> Image.Image:
    """
    Create a side-by-side comparison image.
    """
    from PIL import ImageDraw, ImageFont
    
    # Resize images to same height
    max_height = max(image1.height, image2.height)
    
    # Calculate new dimensions maintaining aspect ratio
    ratio1 = max_height / image1.height
    ratio2 = max_height / image2.height
    
    new_size1 = (int(image1.width * ratio1), max_height)
    new_size2 = (int(image2.width * ratio2), max_height)
    
    img1_resized = image1.resize(new_size1, Image.Resampling.LANCZOS)
    img2_resized = image2.resize(new_size2, Image.Resampling.LANCZOS)
    
    # Create combined canvas
    label_height = 40
    total_width = new_size1[0] + new_size2[0] + padding * 3
    total_height = max_height + label_height + padding * 2
    
    combined = Image.new('RGB', (total_width, total_height), (255, 255, 255))
    
    # Paste images
    combined.paste(img1_resized, (padding, label_height + padding))
    combined.paste(img2_resized, (padding * 2 + new_size1[0], label_height + padding))
    
    # Add labels
    draw = ImageDraw.Draw(combined)
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
    except:
        font = ImageFont.load_default()
    
    draw.text((padding + new_size1[0]//2, 10), labels[0], fill=(0, 0, 0), font=font, anchor="mt")
    draw.text((padding * 2 + new_size1[0] + new_size2[0]//2, 10), labels[1], fill=(0, 0, 0), font=font, anchor="mt")
    
    return combined

# Create and analyze comparison image
comparison_img = create_comparison_image(img1, img2, ("Cat", "Dog"))

plt.figure(figsize=(14, 8))
plt.imshow(comparison_img)
plt.axis('off')
plt.title("Combined Comparison Image")
plt.show()

# Now the VLM can see both!
response = analyze_image(
    comparison_img,
    "This image shows two pictures side by side. Compare the left image and the right image. What are the similarities and differences?",
    max_new_tokens=300
)

print("\n📊 VLM Comparison (seeing both images):")
print(response)

---

## Cleanup

In [ ]:
del model, processor
torch.cuda.empty_cache()
gc.collect()
print("✅ Cleanup complete!")